In [1]:
import os, re

In [2]:
os.chdir('./')

In [36]:
%pwd

'x:\\DL\\zenml\\MLOPs\\zenml'

In [7]:
def get_highest_train_folder(parent_folder):
    items = os.listdir(parent_folder)
    # Filter the folders and get their numbers
    train_folders = sorted(
        [(item, int(item[5:])) for item in items if os.path.isdir(os.path.join(parent_folder, item)) and re.match(r'train\d+$', item)],
        key=lambda x: x[1],
        reverse=True
    )
    # Iterate over the sorted folders from highest to lowest
    for folder, _ in train_folders:
        weights_path = os.path.join(parent_folder, folder, 'weights', 'best.pt')
        if os.path.exists(weights_path):
            return str(folder)
    
    # Return None if no matching folder contains the weights file
    return None

In [10]:
x = get_highest_train_folder('runs/detect')

In [11]:
print(x)

train31


In [4]:
os.environ["MLFLOW_TRACKING_URI"] = "http://127.0.0.1:5000"


In [62]:
import mlflow

# create a client to access the MLflow tracking server
client = mlflow.MlflowClient()

# loop through all registered models
# NOTE: `filter_string` should be optional, but leaving it as `None` failed to work.
# Instead, using `"name LIKE '%'"` will match all model names
for model in client.search_registered_models(filter_string="name LIKE '%'"):
    # loop through the latest versions for each stage of a registered model
    for model_version in model.latest_versions:
        print(
            f"name={model_version.name}; run_id={model_version.run_id}; version={model_version.version}, stage={model_version.current_stage}, source = {model_version.source}"
        )


<ModelVersion: aliases=[], creation_timestamp=1723315785945, current_stage='None', description='', last_updated_timestamp=1723315785945, name='miniv8', run_id='7a6b7b99cea84ddc9299d39c9f448a68', run_link='', source='mlflow-artifacts:/675898299489586662/7a6b7b99cea84ddc9299d39c9f448a68/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='5'>
<ModelVersion: aliases=[], creation_timestamp=1723187670427, current_stage='Archived', description='', last_updated_timestamp=1723307859039, name='miniv8', run_id='3c92b792f47249719a2221bdd332f5ce', run_link='', source='mlflow-artifacts:/675898299489586662/3c92b792f47249719a2221bdd332f5ce/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>
<ModelVersion: aliases=[], creation_timestamp=1723193812025, current_stage='Production', description='', last_updated_timestamp=1723315786710, name='miniv8', run_id='e491156119eb42fbb154e0fd024cf30b', run_link='', source='mlflow-artifacts:/6758982994

In [40]:
def get_registered_model_name_from_run(run_id: str) -> str:
    """
    Given a run ID, return the registered model name associated with that run.

    Args:
        run_id (str): The ID of the run to retrieve the registered model name from.

    Returns:
        str: The name of the registered model associated with the run, or None if no model is found.
    """
    client = mlflow.tracking.MlflowClient()

    # Search for all registered models
    registered_models = client.search_registered_models()

    # Iterate through all registered models
    for model in registered_models:
        # Get all versions of the model
        for version in model.latest_versions:
            # Check if the run ID matches the input run_id
            if version.run_id == run_id:
                return model.name
    
    # If no matching model is found, return None or an appropriate message
    return None

In [41]:
id = "3c92b792f47249719a2221bdd332f5ce"
get_registered_model_name_from_run(id)

'miniv8'

In [42]:
import mlflow

def list_models_and_versions(name=None):
    client = mlflow.tracking.MlflowClient()

    # Get all registered models
    if name:
        registered_models = client.search_registered_models(filter_string="name LIKE '%'")
    
    models_info = {}

    for model in registered_models:
        model_name = model.name
        models_info[model_name] = []

        # Get all versions of the registered model
        versions = model.latest_versions
        
        for version in versions:
            version_info = {
                "version": version.version,
                "run_id": version.run_id,
                "source": version.source
            }
            models_info[model_name].append(version_info)

    return models_info

def get_model_metrics(run_id):
    client = mlflow.tracking.MlflowClient()
    run_data = client.get_run(run_id).data.to_dictionary()
    metrics = {
        "mAP50": run_data['metrics'].get('mAP50'),
        "mAP50-95": run_data['metrics'].get('mAP50-95'),
        "recall": run_data['metrics'].get('recall'),
        "precision": run_data['metrics'].get('precision')
    }
    return metrics

x = "miniv8"

In [43]:
y = list_models_and_versions(x)[x]
print(y)

[{'version': '5', 'run_id': '7a6b7b99cea84ddc9299d39c9f448a68', 'source': 'mlflow-artifacts:/675898299489586662/7a6b7b99cea84ddc9299d39c9f448a68/artifacts/model'}, {'version': '1', 'run_id': '3c92b792f47249719a2221bdd332f5ce', 'source': 'mlflow-artifacts:/675898299489586662/3c92b792f47249719a2221bdd332f5ce/artifacts/model'}, {'version': '2', 'run_id': 'e491156119eb42fbb154e0fd024cf30b', 'source': 'mlflow-artifacts:/675898299489586662/e491156119eb42fbb154e0fd024cf30b/artifacts/model'}]


In [44]:
run_ids = [version['run_id'] and version['version'] for version in y]
print(run_ids)

['5', '1', '2']


In [63]:

def select_best_model(name, thresholds):
    models_info = list_models_and_versions(name)
    
    if name not in models_info:
        return "Model not found."
    
    best_run_id = None
    best_metrics = None
    best_version = None
    
    for version_info in models_info[name]:
        run_id = version_info['run_id']
        metrics = get_model_metrics(run_id)
        
        if all(metrics[metric] >= thresholds[metric] for metric in thresholds):
            if best_metrics is None or (
                metrics['mAP50-95'] > best_metrics['mAP50-95'] or
                (metrics['mAP50-95'] == best_metrics['mAP50-95'] and metrics['mAP50'] > best_metrics['mAP50'])
            ):
                best_run_id = run_id
                best_metrics = metrics
                best_version = version_info['version']
                artifact = version_info['source']
    
    if best_run_id is not None:
        print(f"Best version: {best_version}")
        print(f"Best run ID: {best_run_id}")
        print(f"Best metrics: {best_metrics}")
        print(f"source artifact: {artifact}")
        return best_version, best_run_id, best_metrics, artifact
    else:
        return "No models meet the threshold criteria."
    
# Define the threshold values
thresholds = {
    "mAP50": 0.02,
    "mAP50-95": 0.02,
    "recall": 0.01,
    "precision": 0.002
}

In [64]:
name = 'miniv8'

In [66]:
bmodel,idm,metr,art = select_best_model(name,thresholds)

Best version: 2
Best run ID: e491156119eb42fbb154e0fd024cf30b
Best metrics: {'mAP50': 0.7591, 'mAP50-95': 0.48673, 'recall': 0.69317, 'precision': 0.68404}
source artifact: mlflow-artifacts:/675898299489586662/e491156119eb42fbb154e0fd024cf30b/artifacts/model


In [80]:
art = art.replace("mlflow-artifacts:", "mlartifacts")
print(f"{art}/artifacts/best.pt")

mlartifacts/675898299489586662/e491156119eb42fbb154e0fd024cf30b/artifacts/model/artifacts/best.pt


In [86]:
from typing import Annotated, Any, Dict, Tuple
import mlflow
import logging
#from zenml import get_step_context, step
from zenml.logger import get_logger
from scripts.config.configuration import TresholdMetrics
from dataclasses import asdict
from ultralytics import YOLO

logger = get_logger(__name__)



class ProductionModel:
    def __init__(self) -> None:
        self.client = mlflow.tracking.MlflowClient()

    def get_registered_model_name_from_run(self,run_id: str) -> str:
        # Search for all registered models
        registered_models = self.client.search_registered_models()

        # Iterate through all registered models
        for model in registered_models:
            # Get all versions of the model
            for version in model.latest_versions:
                # Check if the run ID matches the input run_id
                if version.run_id == run_id:
                    logging.info(f"{model.name} is registered under {run_id}")
                    return model.name
                    
        
        # If no matching model is found, return None or an appropriate message
        return None

    def list_models_and_versions(self, name=None):
        # Get all registered models
        if name:
            registered_models = self.client.search_registered_models(filter_string="name LIKE '%'")
        
        models_info = {}

        for model in registered_models:
            model_name = model.name
            models_info[model_name] = []

            # Get all versions of the registered model
            versions = model.latest_versions
            
            for version in versions:
                version_info = {
                    "version": version.version,
                    "run_id": version.run_id,
                    "source": version.source
                }
                models_info[model_name].append(version_info)

        logging.info("model info has been fetched")
        return models_info

    def get_model_metrics(self, run_id):
        
        run_data = self.client.get_run(run_id).data.to_dictionary()
        metrics = {
            "mAP50": run_data['metrics'].get('mAP50'),
            "mAP50_95": run_data['metrics'].get('mAP50-95'),
            # "recall": run_data['metrics'].get('recall'),
            # "precision": run_data['metrics'].get('precision')
        }
        logging.info(f"metrics of {run_id} has been retrieved")
        return metrics
    

#@step   
def production_model(run_id: str,  
                     thresholds:TresholdMetrics) -> Tuple[Annotated[str, "Production model"],
                                                          Annotated[int, "version"]]:

    client = mlflow.tracking.MlflowClient()
    prod = ProductionModel()
    name = prod.get_registered_model_name_from_run(run_id=run_id)
    models_info = prod.list_models_and_versions(name)
        
    if name not in models_info:
        return "Model not found."
        
    best_run_id = None
    best_metrics = None
    best_version = None
       
    for version_info in models_info[name]:
        run_id = version_info['run_id']
        metrics = prod.get_model_metrics(run_id)

        threshold_dict = asdict(thresholds)
            
        if all(metrics[metric] >= threshold_dict[metric] for metric in threshold_dict):
            if best_metrics is None or (
                metrics['mAP50_95'] > best_metrics['mAP50_95'] or
                (metrics['mAP50_95'] == best_metrics['mAP50_95'] and metrics['mAP50'] > best_metrics['mAP50'])
            ):
                best_run_id = run_id
                best_metrics = metrics
                best_version = version_info['version']
                artifact = version_info['source']
                artifact = artifact.replace("mlflow-artifacts:", "mlartifacts")
                model = YOLO(f"{artifact}/artifacts/best.pt")


        
    if best_run_id is not None:
        print(f"Best version: {best_version}")
        print(f"Best run ID: {best_run_id}")
        print(f"Best metrics: {best_metrics}")
        print(f"artifact: {artifact}")
        
        client.transition_model_version_stage(
            name=name,
            version=best_version,
            stage="Production",
            archive_existing_versions=True  # Optional: Archives previous versions in Production
        )
        logging.info(f"{name} model with version{best_version} has been set to production")
        return name, best_version, model
    else:
        logging.info("models failed to meet the threshold, initiate retrain")
        return tuple("No models meet the threshold criteria. kindly update parameters and retrian")

In [87]:
from scripts.config.configuration import ConfigurationManager

x = ConfigurationManager()
run_id = "e491156119eb42fbb154e0fd024cf30b"
threshold = x.get_threshold()

[2024-08-11 02:45:08,821: INFO: common: yaml file: config\config.yaml loaded sucessfully]
yaml file: config\config.yaml loaded sucessfully
[2024-08-11 02:45:08,823: INFO: common: created directory at MLartifacts]
created directory at MLartifacts


In [89]:
x,y,z = production_model(run_id, threshold)
print(type(z))

[2024-08-11 02:45:48,032: INFO: 3705743414: miniv8 is registered under e491156119eb42fbb154e0fd024cf30b]
miniv8 is registered under e491156119eb42fbb154e0fd024cf30b
[2024-08-11 02:45:48,061: INFO: 3705743414: model info has been fetched]
model info has been fetched
[2024-08-11 02:45:48,115: INFO: 3705743414: metrics of 7a6b7b99cea84ddc9299d39c9f448a68 has been retrieved]
metrics of 7a6b7b99cea84ddc9299d39c9f448a68 has been retrieved
[2024-08-11 02:45:48,226: INFO: 3705743414: metrics of 3c92b792f47249719a2221bdd332f5ce has been retrieved]
metrics of 3c92b792f47249719a2221bdd332f5ce has been retrieved
[2024-08-11 02:45:48,279: INFO: 3705743414: metrics of e491156119eb42fbb154e0fd024cf30b has been retrieved]
metrics of e491156119eb42fbb154e0fd024cf30b has been retrieved
Best version: 2
Best run ID: e491156119eb42fbb154e0fd024cf30b
Best metrics: {'mAP50': 0.7591, 'mAP50_95': 0.48673}
artifact: mlartifacts/675898299489586662/e491156119eb42fbb154e0fd024cf30b/artifacts/model
[2024-08-11 02:4

C:\Users\aravi\AppData\Local\Temp\ipykernel_28176\3705743414.py:116: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


In [24]:
version = bmodel[0]
print(version)

2


In [54]:
# Construct the model URI
model_uri = f"models:/{name}/{version}"
print(model_uri)
# Load the model
model = mlflow.pyfunc.load_model(model_uri)



models:/miniv8/2


[2024-08-11 01:33:46,488: INFO: wrapper: artifacts[path]: runs/detect/train47/weights/best.pt]
artifacts[path]: runs/detect/train47/weights/best.pt


In [27]:
model_name = name
model_version = version


# Transition the model version to the Production stage
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage="Production",
    archive_existing_versions=True  # Optional: Archives previous versions in Production
)


C:\Users\aravi\AppData\Local\Temp\ipykernel_28176\3643407718.py:6: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1723193812025, current_stage='Production', description='', last_updated_timestamp=1723300226016, name='miniv8', run_id='e491156119eb42fbb154e0fd024cf30b', run_link='', source='mlflow-artifacts:/675898299489586662/e491156119eb42fbb154e0fd024cf30b/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='2'>

In [51]:
import os
from PIL import Image

def run_inference(model, image_directory, save_txt=True):
    # List all image files in the directory
    image_files = [os.path.join(image_directory, img) for img in os.listdir(image_directory) if img.endswith(('.jpg', '.jpeg', '.png'))]

    for image_path in image_files:
        # Open the image to determine its size
        with Image.open(image_path) as img:
            width, height = img.size
            imgsz = max(width, height)  # Use the larger dimension as the input size

        # Prepare the input dictionary for each image
        data = {
            "source": image_path,
            "imgsz": imgsz,
            "save_txt": save_txt,
        }

        # Run the model on the current image
        results = model.predict(data=data)  # Assuming 'model' is a callable that accepts a dictionary as input
        print(results)
        # Process or save results as needed


In [7]:
from ultralytics import YOLO


model = YOLO("MLartifacts/675898299489586662/e491156119eb42fbb154e0fd024cf30b/artifacts/model/artifacts/best.pt")


In [114]:
model.predict("MLartifacts\\data_ingestion\\v2\\\images\\", save=True, save_txt = True)


image 1/10 x:\DL\zenml\MLOPs\zenml\MLartifacts\data_ingestion\v2\images\1463_jpeg.rf.d086eddf1f8191761babb1f3831214c6.jpg: 640x640 2 Drills, 194.4ms
image 2/10 x:\DL\zenml\MLOPs\zenml\MLartifacts\data_ingestion\v2\images\1464_jpeg.rf.883944bd5d32085dd7ffb1c83e631395.jpg: 640x640 1 Drill, 94.1ms
image 3/10 x:\DL\zenml\MLOPs\zenml\MLartifacts\data_ingestion\v2\images\1465_jpeg.rf.e9f3fae8fef7ffa7248ade1764e44299.jpg: 640x640 1 Drill, 100.7ms
image 4/10 x:\DL\zenml\MLOPs\zenml\MLartifacts\data_ingestion\v2\images\1466_jpeg.rf.43197d898c4b43ef5fc0cf05255c72d3.jpg: 640x640 2 Drills, 111.1ms
image 5/10 x:\DL\zenml\MLOPs\zenml\MLartifacts\data_ingestion\v2\images\1467_jpeg.rf.932e89684cfcbd87534cc3e5b699cf5a.jpg: 640x640 1 Drill, 102.5ms
image 6/10 x:\DL\zenml\MLOPs\zenml\MLartifacts\data_ingestion\v2\images\1468_jpeg.rf.0c72eb06a8539d6ea14d8ff45cb90fa0.jpg: 640x640 1 Drill, 100.0ms
image 7/10 x:\DL\zenml\MLOPs\zenml\MLartifacts\data_ingestion\v2\images\1468_jpeg.rf.b71a410026731c8242ec4fb60

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'Drill', 1: 'Survivor'}
 obb: None
 orig_img: array([[[  0,   0,  43],
         [ 35, 116, 161],
         [ 27, 126, 176],
         ...,
         [154, 181, 215],
         [157, 182, 214],
         [ 38,  63,  95]],
 
        [[  0,   0,  43],
         [ 36, 114, 160],
         [ 24, 123, 173],
         ...,
         [157, 182, 216],
         [159, 184, 216],
         [ 35,  60,  92]],
 
        [[  0,   2,  45],
         [ 34, 112, 158],
         [ 20, 119, 169],
         ...,
         [162, 184, 219],
         [163, 186, 218],
         [ 32,  56,  86]],
 
        ...,
 
        [[ 27,  17, 137],
         [ 38,  29, 145],
         [ 40,  34, 143],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],
 
        [[ 31,  21, 135],
         [ 38,  29, 139],
         [ 44,  38, 139],
         ...,
    

In [75]:

image_directory="MLartifacts/data_ingestion/version1/test/"
image_files = [os.path.join(image_directory, img) for img in os.listdir(image_directory) if img.endswith(('.jpg', '.jpeg', '.png'))]

for image_path in image_files:
        # Open the image to determine its size
    with Image.open(image_path) as img:
        width, height = img.size
        imgsz = max(width, height)  # Use the larger dimension as the input size

        # Prepare the input dictionary for each image
    data = {
        "source": image_path,
        "imgsz": imgsz,
        "save_txt": True,
        }

        # Run the model on the current image
    results = model.predict(data=data)  # Assuming 'model' is a callable that accepts a dictionary as input

In [50]:
data = {
    "source": "X:\\DL\\zenml\\MLOPs\\zenml\\MLartifacts\\data_ingestion\\version1\\train\\images\\1130_jpeg.rf.6f4cc78e52b0db78c3149a77ac819f7d.jpg",
    "imgsz": "640",
    "save_txt": "True",
}


results = model.predict(data=data)
print(results)

[2024-08-11 01:23:42,473: INFO: wrapper: Data input: f{'source': 'X:\\DL\\zenml\\MLOPs\\zenml\\MLartifacts\\data_ingestion\\version1\\train\\images\\1130_jpeg.rf.6f4cc78e52b0db78c3149a77ac819f7d.jpg', 'imgsz': '640', 'save_txt': 'True'}]
Data input: f{'source': 'X:\\DL\\zenml\\MLOPs\\zenml\\MLartifacts\\data_ingestion\\version1\\train\\images\\1130_jpeg.rf.6f4cc78e52b0db78c3149a77ac819f7d.jpg', 'imgsz': '640', 'save_txt': 'True'}
[2024-08-11 01:23:42,477: INFO: wrapper: Data after reformat: f{'source': 'X:\\DL\\zenml\\MLOPs\\zenml\\MLartifacts\\data_ingestion\\version1\\train\\images\\1130_jpeg.rf.6f4cc78e52b0db78c3149a77ac819f7d.jpg', 'imgsz': 640, 'save_txt': True}]
Data after reformat: f{'source': 'X:\\DL\\zenml\\MLOPs\\zenml\\MLartifacts\\data_ingestion\\version1\\train\\images\\1130_jpeg.rf.6f4cc78e52b0db78c3149a77ac819f7d.jpg', 'imgsz': 640, 'save_txt': True}

image 1/1 X:\DL\zenml\MLOPs\zenml\MLartifacts\data_ingestion\version1\train\images\1130_jpeg.rf.6f4cc78e52b0db78c3149a77a

In [106]:
from zenml.steps import step
import torch
import os
import numpy as np
from PIL import Image
import torchvision.transforms as transforms
from sklearn.metrics import precision_recall_curve, average_precision_score

#@step
def inference_step(model, folder_directory: str, threshold: float = 0.5):
    images_folder = os.path.join(folder_directory, 'images')
    labels_folder = os.path.join(folder_directory, 'labels')
    
    all_predictions = []
    all_ground_truths = []
    
    transform = transforms.Compose([transforms.ToTensor()])

    for image_file in os.listdir(images_folder):
        image_path = os.path.join(images_folder, image_file)
        label_file = os.path.splitext(image_file)[0] + '.txt'
        label_path = os.path.join(labels_folder, label_file)

        # Load image and labels
        img = Image.open(image_path).convert('RGB')
        img = transform(img).unsqueeze(0)  # Convert image to tensor and add batch dimension
        
        ground_truth = np.loadtxt(label_path) if os.path.exists(label_path) else None
        
        # Perform inference
        results = model(img)
        
        # Process model output
        if len(results) == 0 or not hasattr(results[0], 'boxes'):
            print(f"No detections for image: {image_file}")
            continue
        
        boxes = results[0].boxes.xyxy.cpu().numpy()  # Get bounding boxes in xyxy format
        scores = results[0].boxes.conf.cpu().numpy()  # Get confidence scores
        classes = results[0].boxes.cls.cpu().numpy()  # Get class labels
        
        predictions = {
            "classes": classes.tolist(),
            "boxes": boxes.tolist(),
            "scores": scores.tolist(),
        }
        all_predictions.append(predictions)
        
        if ground_truth is not None:
            all_ground_truths.append(ground_truth)
    
    # Calculate mAP50 (for simplicity, here we just return precision as a placeholder)
    metrics = {}
    aps = []
    
    for cls in np.unique(classes):
        cls_ground_truths = [gt for gt, cls_gt in zip(all_ground_truths, classes) if cls_gt == cls]
        cls_detections = [det for det, cls_det in zip(all_predictions, classes) if cls_det == cls]
        precision, recall, _ = precision_recall_curve(cls_ground_truths, cls_detections)
        ap = average_precision_score(cls_ground_truths, cls_detections)
        aps.append(ap)
    
    metrics['mAP50'] = np.mean(aps)
    performance = metrics['mAP50']
    
    if performance < threshold:
        # Trigger retrain if performance is below the threshold
        print("Performance below threshold. Initiating retraining...")
        # Add retraining logic here
    
    return {"metrics": metrics, "performance": performance, "trigger_retrain": performance < threshold}

In [18]:
import os
import cv2
from zenml.steps import step
from typing import List, Tuple
from ultralytics import YOLO  # Assuming the YOLOv8 model is from ultralytics

#@step
def yolov8_inference_step(
    model: YOLO, 
    images_folder: str, 
    save_images: bool = True, 
    save_labels: bool = True, 
    output_folder: str = "output"
) -> List[Tuple[str, List[Tuple[int, float, Tuple[float, float, float, float]]]]]:
    """Performs inference using the provided YOLOv8 model and returns class, bounding box, and confidence score.
    
    Args:
        model (YOLO): Preloaded YOLOv8 model.
        images_folder (str): Folder containing images for inference.
        save_images (bool): Whether to save output images with bounding boxes drawn.
        save_labels (bool): Whether to save label files with predictions.
        output_folder (str): Directory to save output images and labels.
        
    Returns:
        List[Tuple[str, List[Tuple[int, float, Tuple[float, float, float, float]]]]]: List of tuples containing 
            image filename and a list of tuples with class, confidence score, and bounding box coordinates.
    """
    
    results = []
    
    # Create output directories
    os.makedirs(output_folder, exist_ok=True)
    if save_labels:
        labels_folder = os.path.join(output_folder, "labels")
        os.makedirs(labels_folder, exist_ok=True)
    if save_images:
        images_output_folder = os.path.join(output_folder, "images")
        os.makedirs(images_output_folder, exist_ok=True)
    
    for image_file in os.listdir(images_folder):
        if image_file.endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(images_folder, image_file)
            img = cv2.imread(image_path)
            
            # Perform inference using the preloaded model
            predictions = model(img)  # Get predictions
            
            results_per_image = []
            for pred in predictions[0]:
                class_id, confidence, bbox = int(pred[5]), pred[4], (pred[0], pred[1], pred[2], pred[3])
                results_per_image.append((class_id, confidence, bbox))
            
            results.append((image_file, results_per_image))
            
            # Save labels
            if save_labels:
                label_file = os.path.join(labels_folder, f"{os.path.splitext(image_file)[0]}.txt")
                with open(label_file, 'w') as f:
                    for class_id, confidence, bbox in results_per_image:
                        x_center, y_center, width, height = bbox
                        f.write(f"{class_id} {x_center} {y_center} {width} {height} {confidence}\n")
            
            # Draw bounding boxes and save images
            if save_images:
                for class_id, confidence, bbox in results_per_image:
                    x_center, y_center, width, height = bbox
                    x1, y1 = int(x_center - width / 2), int(y_center - height / 2)
                    x2, y2 = int(x_center + width / 2), int(y_center + height / 2)
                    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(img, f"{class_id} {confidence:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                
                output_image_path = os.path.join(images_output_folder, image_file)
                cv2.imwrite(output_image_path, img)
    
    return results


In [19]:
yolov8_inference_step(model=model,
                      images_folder="MLartifacts/data_ingestion/v2/images",
                      output_folder="MLartifacts/predictions/v2")


0: 640x640 2 Drills, 121.1ms
Speed: 8.0ms preprocess, 121.1ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 yolov8_inference_step(model=model,                                                           │
│   2 │   │   │   │   │     images_folder="MLartifacts/data_ingestion/v2/images",                  │
│   3 │   │   │   │   │     output_folder="MLartifacts/predictions/v2")                            │
│   4                                                                                              │
│                                                                                                  │
│ in yolov8_inference_step:50                                                                      │
│                                                                                                  │
│   47 │   │   │                                                                                   │
│   48 │   │   │   results_per_image = []                                                          │
│   49 │   │   │   for pred in predictions[0]:                                                     │
│ ❱ 50 │   │   │   │   class_id, confidence, bbox = int(pred[5]), pred[4], (pred[0], pred[1], p    │
│   51 │   │   │   │   results_per_image.append((class_id, confidence, bbox))                      │
│   52 │   │   │                                                                                   │
│   53 │   │   │   results.append((image_file, results_per_image))                                 │
│                                                                                                  │
│ c:\Users\aravi\anaconda3\envs\zenml\lib\site-packages\ultralytics\engine\results.py:287 in       │
│ __getitem__                                                                                      │
│                                                                                                  │
│    284 │   │   │   >>> single_result = results[0]  # Get the first result                        │
│    285 │   │   │   >>> subset_results = results[1:4]  # Get a slice of results                   │
│    286 │   │   """                                                                               │
│ ❱  287 │   │   return self._apply("__getitem__", idx)                                            │
│    288 │                                                                                         │
│    289 │   def __len__(self):                                                                    │
│    290 │   │   """                                                                               │
│                                                                                                  │
│ c:\Users\aravi\anaconda3\envs\zenml\lib\site-packages\ultralytics\engine\results.py:359 in       │
│ _apply                                                                                           │
│                                                                                                  │
│    356 │   │   for k in self._keys:                                                              │
│    357 │   │   │   v = getattr(self, k)                                                          │
│    358 │   │   │   if v is not None:                                                             │
│ ❱  359 │   │   │   │   setattr(r, k, getattr(v, fn)(*args, **kwargs))                            │
│    360 │   │   return r                                                                          │
│    361 │                                                                                         │
│    362 │   def cpu(self):                                                                        │
│                                                            

In [9]:
inference_step(model,"MLartifacts/data_ingestion/v2")


image 1/1 x:\DL\zenml\MLOPs\zenml\MLartifacts\data_ingestion\v2\images\1463_jpeg.rf.d086eddf1f8191761babb1f3831214c6.jpg: 640x640 2 Drills, 143.4ms
Speed: 3.6ms preprocess, 143.4ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 x:\DL\zenml\MLOPs\zenml\MLartifacts\data_ingestion\v2\images\1464_jpeg.rf.883944bd5d32085dd7ffb1c83e631395.jpg: 640x640 1 Drill, 83.1ms
Speed: 0.0ms preprocess, 83.1ms inference, 5.3ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 x:\DL\zenml\MLOPs\zenml\MLartifacts\data_ingestion\v2\images\1465_jpeg.rf.e9f3fae8fef7ffa7248ade1764e44299.jpg: 640x640 1 Drill, 88.2ms
Speed: 0.0ms preprocess, 88.2ms inference, 8.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 x:\DL\zenml\MLOPs\zenml\MLartifacts\data_ingestion\v2\images\1466_jpeg.rf.43197d898c4b43ef5fc0cf05255c72d3.jpg: 640x640 2 Drills, 78.6ms
Speed: 0.0ms preprocess, 78.6ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 x:\DL\zen

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 inference_step(model,"MLartifacts/data_ingestion/v2")                                        │
│   2                                                                                              │
│                                                                                                  │
│ in inference_step:64                                                                             │
│                                                                                                  │
│   61 │   │   annotated_images.append(annotated_image_path)                                       │
│   62 │                                                                                           │
│   63 │   # Compute mAP                                                                           │
│ ❱ 64 │   map_dict = metric.compute()                                                             │
│   65 │   mAP50 = map_dict['map_50'].item()                                                       │
│   66 │                                                                                           │
│   67 │   if mAP50 < threshold:                                                                   │
│                                                                                                  │
│ c:\Users\aravi\anaconda3\envs\zenml\lib\site-packages\torchmetrics\metric.py:633 in wrapped_func │
│                                                                                                  │
│    630 │   │   │   │   should_sync=self._to_sync,                                                │
│    631 │   │   │   │   should_unsync=self._should_unsync,                                        │
│    632 │   │   │   ):                                                                            │
│ ❱  633 │   │   │   │   value = _squeeze_if_scalar(compute(*args, **kwargs))                      │
│    634 │   │   │   │   # clone tensor to avoid in-place operations after compute, altering alre  │
│    635 │   │   │   │   value = apply_to_collection(value, Tensor, lambda x: x.clone())           │
│    636                                                                                           │
│                                                                                                  │
│ c:\Users\aravi\anaconda3\envs\zenml\lib\site-packages\torchmetrics\detection\mean_ap.py:515 in   │
│ compute                                                                                          │
│                                                                                                  │
│    512 │                                                                                         │
│    513 │   def compute(self) -> dict:                                                            │
│    514 │   │   """Computes the metric."""                                                        │
│ ❱  515 │   │   coco_preds, coco_target = self._get_coco_datasets(average=self.average)           │
│    516 │   │                                                                                     │
│    517 │   │   result_dict = {}                                                                  │
│    518 │   │   with contextlib.redirect_stdout(io.StringIO()):                                   │
│                                                                                                  │
│ c:\Users\aravi\anaconda3\envs\zenml\lib\site-packages\torchmetrics\detection\mean_ap.py:602 in   │
│ _get_coco_datasets                                                                               │
│                                                            

In [2]:
from zenml.client import Client
from zenml.enums import ExecutionStatus

pipeline_model = Client().get_pipeline("data_pipeline")
print(pipeline_model)

body=PipelineResponseBody(created=datetime.datetime(2024, 8, 7, 6, 42, 21, 576456), updated=datetime.datetime(2024, 8, 7, 6, 42, 21, 576456), user=UserResponse(body=UserResponseBody(created=datetime.datetime(2024, 8, 6, 6, 23, 59, 703025), updated=datetime.datetime(2024, 8, 6, 7, 21, 46, 234316), active=True, activation_token=None, full_name='', email_opted_in=True, is_service_account=False, is_admin=True), metadata=None, resources=None, id=UUID('8f6a528b-1bdf-4cb8-bd37-c1879db69d58'), permission_denied=False, name='default'), latest_run_id=UUID('c6a174b7-bd23-4949-98d6-85596e7f0d4b'), latest_run_status=<ExecutionStatus.RUNNING: 'running'>) metadata=PipelineResponseMetadata(workspace=WorkspaceResponse(body=WorkspaceResponseBody(created=datetime.datetime(2024, 8, 6, 6, 23, 59, 283844), updated=datetime.datetime(2024, 8, 6, 6, 23, 59, 283844)), metadata=None, resources=None, id=UUID('b107c1b0-efb8-4b44-b8cf-d50229da73df'), permission_denied=False, name='default'), description=None) resou

In [3]:
runs = pipeline_model.runs[-1]
print(runs)

body=PipelineRunResponseBody(created=datetime.datetime(2024, 8, 18, 17, 34, 57, 679947), updated=datetime.datetime(2024, 8, 18, 17, 34, 57, 793221), user=UserResponse(body=UserResponseBody(created=datetime.datetime(2024, 8, 6, 6, 23, 59, 703025), updated=datetime.datetime(2024, 8, 6, 7, 21, 46, 234316), active=True, activation_token=None, full_name='', email_opted_in=True, is_service_account=False, is_admin=True), metadata=None, resources=None, id=UUID('8f6a528b-1bdf-4cb8-bd37-c1879db69d58'), permission_denied=False, name='default'), status=<ExecutionStatus.RUNNING: 'running'>, stack=StackResponse(body=StackResponseBody(created=datetime.datetime(2024, 8, 7, 5, 5, 13, 595496), updated=datetime.datetime(2024, 8, 7, 5, 5, 13, 595496), user=UserResponse(body=UserResponseBody(created=datetime.datetime(2024, 8, 6, 6, 23, 59, 703025), updated=datetime.datetime(2024, 8, 6, 7, 21, 46, 234316), active=True, activation_token=None, full_name='', email_opted_in=True, is_service_account=False, is_ad

In [12]:
dataset = Client().get_artifact_version("Dataset_path")
val_status_artifact = Client().get_artifact_version("Retrain_trigger")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 dataset = Client().get_artifact_version("Dataset_path")                                      │
│ ❱ 2 val_status_artifact = Client().get_artifact_version("Retrain_trigger")                       │
│   3                                                                                              │
│                                                                                                  │
│ c:\Users\aravi\anaconda3\envs\zenml\lib\site-packages\zenml\client_lazy_loader.py:198 in _inner  │
│                                                                                                  │
│   195 │   │   │   │   │   with contextlib.suppress(ValueError):                                  │
│   196 │   │   │   │   │   │   kwargs[k] = ClientLazyLoader(**v).evaluate()                       │
│   197 │   │   │                                                                                  │
│ ❱ 198 │   │   │   return func(*args_, **kwargs)                                                  │
│   199 │   │                                                                                      │
│   200 │   │   return _inner                                                                      │
│   201                                                                                            │
│                                                                                                  │
│ c:\Users\aravi\anaconda3\envs\zenml\lib\site-packages\zenml\client.py:4122 in                    │
│ get_artifact_version                                                                             │
│                                                                                                  │
│   4119 │   │   │   hydrate=hydrate,                                                              │
│   4120 │   │   ):                                                                                │
│   4121 │   │   │   return cll  # type: ignore[return-value]                                      │
│ ❱ 4122 │   │   return self._get_entity_version_by_id_or_name_or_prefix(                          │
│   4123 │   │   │   get_method=self.zen_store.get_artifact_version,                               │
│   4124 │   │   │   list_method=self.list_artifact_versions,                                      │
│   4125 │   │   │   name_id_or_prefix=name_id_or_prefix,                                          │
│                                                                                                  │
│ c:\Users\aravi\anaconda3\envs\zenml\lib\site-packages\zenml\client_lazy_loader.py:198 in _inner  │
│                                                                                                  │
│   195 │   │   │   │   │   with contextlib.suppress(ValueError):                                  │
│   196 │   │   │   │   │   │   kwargs[k] = ClientLazyLoader(**v).evaluate()                       │
│   197 │   │   │                                                                                  │
│ ❱ 198 │   │   │   return func(*args_, **kwargs)                                                  │
│   199 │   │                                                                                      │
│   200 │   │   return _inner                                                                      │
│   201                                                                                            │
│                                                                                                  │
│ c:\Users\aravi\anaconda3\envs\zenml\lib\site-packages\zenml\client.py:6911 in                    │
│ _get_entity_version_by_id_or_name_or_prefix                

In [11]:
from zenml.materializers import BuiltInMaterializer


mode = dataset.uri

materializer_class = BuiltInMaterializer
status_materializer = materializer_class(dataset)


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:7                                                                                    │
│                                                                                                  │
│   4 mode = dataset.uri                                                                           │
│   5                                                                                              │
│   6 materializer_class = BuiltInMaterializer                                                     │
│ ❱ 7 status_materializer = materializer_class(dataset)                                            │
│   8                                                                                              │
│                                                                                                  │
│ c:\Users\aravi\anaconda3\envs\zenml\lib\site-packages\zenml\materializers\built_in_materializer. │
│ py:69 in __init__                                                                                │
│                                                                                                  │
│    66 │   │   │   artifact_store: The artifact store where the artifact data is stored.          │
│    67 │   │   """                                                                                │
│    68 │   │   super().__init__(uri, artifact_store)                                              │
│ ❱  69 │   │   self.data_path = os.path.join(self.uri, DEFAULT_FILENAME)                          │
│    70 │                                                                                          │
│    71 │   def load(                                                                              │
│    72 │   │   self, data_type: Union[Type[bool], Type[float], Type[int], Type[str]]              │
│                                                                                                  │
│ c:\Users\aravi\anaconda3\envs\zenml\lib\ntpath.py:78 in join                                     │
│                                                                                                  │
│    75                                                                                            │
│    76 # Join two (or more) paths.                                                                │
│    77 def join(path, *paths):                                                                    │
│ ❱  78 │   path = os.fspath(path)                                                                 │
│    79 │   if isinstance(path, bytes):                                                            │
│    80 │   │   sep = b'\\'                                                                        │
│    81 │   │   seps = b'\\/'                                                                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: expected str, bytes or os.PathLike object, not ArtifactVersionResponse

In [10]:
status_materializer.load(str)

'Data\\data_ingestion\\version1_v3'

In [91]:
materializer_instance = materializer_class(dataset.uri)
loaded_data = materializer_instance.load(str)
print(loaded_data)

MLartifacts\data_ingestion\version1


In [79]:
model = Client().get_artifact_version("Production_YOLO")

In [84]:
materializer_instance = materializer_class(model.uri)
loaded = materializer_instance.load(model.uri)
print(loaded)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 materializer_instance = materializer_class(model.uri)                                        │
│   2 loaded = materializer_instance.load(model.uri)                                               │
│   3 print(loaded)                                                                                │
│   4                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: 'BuiltInMaterializer' object is not callable

In [41]:
prediction_step = Client().get_artifact_version("Production_YOLO")

print(type(prediction_step))

<class 'zenml.models.v2.core.artifact_version.ArtifactVersionResponse'>


In [42]:
model = Client().get_artifact_version("Production_YOLO")

In [43]:
mo

body=ArtifactVersionResponseBody(created=datetime.datetime(2024, 8, 11, 19, 5, 31, 780396), updated=datetime.datetime(2024, 8, 11, 19, 5, 31, 780396), user=UserResponse(body=UserResponseBody(created=datetime.datetime(2024, 8, 6, 6, 23, 59, 703025), updated=datetime.datetime(2024, 8, 6, 7, 21, 46, 234316), active=True, activation_token=None, full_name='', email_opted_in=True, is_service_account=False, is_admin=True), metadata=None, resources=None, id=UUID('8f6a528b-1bdf-4cb8-bd37-c1879db69d58'), permission_denied=False, name='default'), artifact=ArtifactResponse(body=ArtifactResponseBody(created=datetime.datetime(2024, 8, 10, 21, 30, 13, 753723), updated=datetime.datetime(2024, 8, 10, 21, 30, 13, 753723), tags=[], latest_version_name='3', latest_version_id=UUID('73c45add-7c6c-4844-8f58-eb01b96ab5f4')), metadata=None, resources=None, id=UUID('bdf9e553-e333-48a4-ae24-78857dfbeb50'), permission_denied=False, name='Production_YOLO'), version='3', uri='X:/DL/zenml/MLOPs/zenml/artifacts/produ

In [92]:
#extract tarfile

import tarfile
import os

def untar(source, out):
    os.makedirs(out, exist_ok=True)
    with tarfile.open(source, 'r') as tarr:
        tarr.extractall(path=out)



In [93]:
out = 'X:/DL/dataset'
untar("C:/Users/aravi/Downloads/part1.tar.gz",out)

In [94]:
untar("C:/Users/aravi/Downloads/part2.tar.gz", out)

In [95]:
untar("C:/Users/aravi/Downloads/part3.tar.gz", out)

In [101]:
def replace_first_element_in_txt(directory_path, new_value):
    for filename in os.listdir(directory_path):
        if filename.endswith(".txt"):  # Process only .txt files
            file_path = os.path.join(directory_path, filename)
            with open(file_path, 'r') as file:
                lines = file.readlines()

            updated_lines = []
            for line in lines:
                elements = line.strip().split()
                if elements:
                    elements[0] = str(new_value)  # Replace the first element with the new value
                    updated_lines.append(" ".join(elements) + "\n")

            # Save the modified content back to the same file
            with open(file_path, 'w') as file:
                file.writelines(updated_lines)

In [102]:
train = "C:\\Users\\aravi\\Downloads\\child detection.v3i.yolov8\\train\\labels"
valid = "C:\\Users\\aravi\\Downloads\\child detection.v3i.yolov8\\valid\\labels"
test = "C:\\Users\\aravi\\Downloads\\child detection.v3i.yolov8\\test\\labels"
ntrain = "C:\\Users\\aravi\\Downloads\\child detection.v3i.yolov8\\train\\label"
nvalid = "C:\\Users\\aravi\\Downloads\\child detection.v3i.yolov8\\valid\\label"
ntest = "C:\\Users\\aravi\\Downloads\\child detection.v3i.yolov8\\test\\label"

In [105]:
replace_first_element_in_txt(test, 2)

In [11]:
from zenml.client import Client
from zenml import load_artifact

from zenml import save_artifact, load_artifact

In [19]:
load_artifact("Dataset_path")

'Data\\data_ingestion\\version1'

In [5]:
last_run = client.get_pipeline("data_pipeline").last_successful_run


In [2]:
class sam:
    def __init__(self, a, b):
        self.a = a
        self.b = b

x = sam("x")

TypeError: __init__() missing 1 required positional argument: 'b'

In [16]:
from ultralytics import YOLO

model = YOLO("/home/arvind/Downloads/coco1.pt")
model.predict("/home/arvind/Python/kubeflow/test/test set", conf = 0.3, save = True)


image 1/381 /home/arvind/Python/kubeflow/test/test set/0_9z848LraIppKIjUK.jpg: 448x640 1 male, 2 females, 324.2ms
image 2/381 /home/arvind/Python/kubeflow/test/test set/1000_F_499480695_gs0cZXiHg3N6107dIRZfcPUzXF4ecljr.jpg: 448x640 5 females, 318.1ms
image 3/381 /home/arvind/Python/kubeflow/test/test set/12-MALE-PERSONALITY-TYPES-Which-One-Are-You.jpg: 448x640 12 males, 319.5ms
image 4/381 /home/arvind/Python/kubeflow/test/test set/1647355906066.jpeg: 448x640 3 females, 331.8ms
image 5/381 /home/arvind/Python/kubeflow/test/test set/1920_e-bus-man-busworld2019-968447.jpg: 416x640 4 buss, 322.1ms
image 6/381 /home/arvind/Python/kubeflow/test/test set/1_qre-gAVNTuazaUPvNw2w-Q.jpg: 448x640 (no detections), 328.8ms
image 7/381 /home/arvind/Python/kubeflow/test/test set/2-3.png: 480x640 1 car, 356.9ms
image 8/381 /home/arvind/Python/kubeflow/test/test set/2010-kodiak-bear-1.jpg: 448x640 1 bear, 337.7ms
image 9/381 /home/arvind/Python/kubeflow/test/test set/210224181151nakayacarcupc01.png: 5

libpng warning: iCCP: known incorrect sRGB profile


image 67/381 /home/arvind/Python/kubeflow/test/test set/csumcwiscedugraphicspedestrianbridgefans201.png: 640x448 5 persons, 298.9ms
image 68/381 /home/arvind/Python/kubeflow/test/test set/customcruisersnovaorgGlenn20Taylor27s20Other20Car.png: 352x640 1 truck, 260.1ms
image 69/381 /home/arvind/Python/kubeflow/test/test set/dafgshdjf.jpeg: 352x640 6 sheeps, 232.0ms
image 70/381 /home/arvind/Python/kubeflow/test/test set/dbfngfgf.jpeg: 640x448 1 giraffe, 297.2ms
image 71/381 /home/arvind/Python/kubeflow/test/test set/dbrshw.jpeg: 640x640 2 cars, 1 bus, 3 trucks, 429.4ms
image 72/381 /home/arvind/Python/kubeflow/test/test set/dbsgw.jpg: 480x640 1 car, 2 trucks, 312.3ms
image 73/381 /home/arvind/Python/kubeflow/test/test set/ddd17ddad60e6d0024dc6144c3c721d5.jpg: 640x512 2 cats, 380.9ms
image 74/381 /home/arvind/Python/kubeflow/test/test set/deptswashingtoneduhiprcimagesrotatePedestrian.png: 640x416 1 male, 1 female, 275.9ms
image 75/381 /home/arvind/Python/kubeflow/test/test set/dfbeb.jpeg:

libpng warning: iCCP: known incorrect sRGB profile


image 186/381 /home/arvind/Python/kubeflow/test/test set/person_120.png: 640x480 1 male, 1 female, 369.2ms
image 187/381 /home/arvind/Python/kubeflow/test/test set/person_127.png: 480x640 1 car, 1 male, 1 female, 4 persons, 236.8ms


...

libpng warning: iCCP: known incorrect sRGB profile


image 366/381 /home/arvind/Python/kubeflow/test/test set/wwwsturgeonbaypolicecomgraphicsbethbikepatrol1.png: 480x640 1 bicycle, 6 cars, 1 male, 311.1ms
image 367/381 /home/arvind/Python/kubeflow/test/test set/wwwsunsetcyclescomPicture20GalleryCustomers20BikesNew20Bike204.png: 480x640 1 bicycle, 340.2ms
image 368/381 /home/arvind/Python/kubeflow/test/test set/wwwsunsetcyclescomPicture20GalleryCustomers20Bikesbradyand20bike.png: 480x640 1 bicycle, 1 female, 313.4ms
image 369/381 /home/arvind/Python/kubeflow/test/test set/wwwtamegoeswildcomphotosjapan200112imagesBiketrailer.png: 448x640 1 bicycle, 1 male, 1 female, 290.3ms
image 370/381 /home/arvind/Python/kubeflow/test/test set/wwwthirdwavewebsitescombikethreeafter.png: 480x640 2 bicycles, 2 cars, 3 males, 304.8ms
image 371/381 /home/arvind/Python/kubeflow/test/test set/wwwuncedualanjimagesMy20New20Seven20Bike.png: 480x640 1 bicycle, 296.8ms
image 372/381 /home/arvind/Python/kubeflow/test/test set/wwwuncoedulibraryarchivesArchives20image

libpng warning: iCCP: known incorrect sRGB profile


image 379/381 /home/arvind/Python/kubeflow/test/test set/yahoohyperpermiscommotoephorscir4022.png: 448x640 1 motorcycle, 2 males, 1 person, 294.1ms
image 380/381 /home/arvind/Python/kubeflow/test/test set/yrtj.jpg: 384x640 11 cows, 268.1ms
image 381/381 /home/arvind/Python/kubeflow/test/test set/zeshs.jpg: 640x544 5 females, 352.7ms
Speed: 1.7ms preprocess, 308.1ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 544)
Results saved to runs/detect/predict


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'bicycle', 1: 'car', 2: 'motorcycle', 3: 'bus', 4: 'truck', 5: 'cat', 6: 'dog', 7: 'horse', 8: 'sheep', 9: 'cow', 10: 'elephant', 11: 'bear', 12: 'zebra', 13: 'giraffe', 14: 'kid', 15: 'male', 16: 'female', 17: 'Numberplate', 18: 'face', 19: 'autorikshaw', 20: 'b', 21: 'c', 22: 'd', 23: 'e', 24: 'person'}
 obb: None
 orig_img: array([[[179, 185, 180],
         [179, 185, 180],
         [179, 185, 180],
         ...,
         [186, 189, 187],
         [185, 188, 186],
         [183, 186, 184]],
 
        [[179, 185, 180],
         [179, 185, 180],
         [179, 185, 180],
         ...,
         [186, 189, 187],
         [185, 188, 186],
         [183, 186, 184]],
 
        [[179, 185, 180],
         [179, 185, 180],
         [179, 185, 180],
         ...,
         [186, 189, 187],
         [185, 188, 186],
         [183, 186, 184]],
 
 

In [12]:
model.names

{0: 'bicycle',
 1: 'car',
 2: 'motorcycle',
 3: 'bus',
 4: 'truck',
 5: 'cat',
 6: 'dog',
 7: 'horse',
 8: 'sheep',
 9: 'cow',
 10: 'elephant',
 11: 'bear',
 12: 'zebra',
 13: 'giraffe',
 14: 'kid',
 15: 'male',
 16: 'female',
 17: 'Numberplate',
 18: 'face',
 19: 'autorikshaw',
 20: 'b',
 21: 'c',
 22: 'd',
 23: 'e',
 24: 'person'}

In [23]:
def get_images_in_path(directory):
    """
    Returns a list of tuples containing the image file names and their full paths
    from the specified directory. It includes both .png and .jpg files.
    
    :param directory: Path to the directory where images are located
    :return: List of tuples (image_name, full_path)
    """
    path = Path(directory)
    image_files = list(path.glob('**/*.[pj][pn]g'))  # Search for .png and .jpg files recursively
    
    # Create a list of tuples (image_name, full_path)
    images_list = [(image_file.name, str(image_file)) for image_file in image_files]
    
    return images_list


images = get_images_in_path("runs/detect/train12")
for image_name, image_path in images:
    print(f"Image Name: {image_name}, Full Path: {image_path}")

Image Name: F1_curve.png, Full Path: runs/detect/train12/F1_curve.png
Image Name: train_batch0.jpg, Full Path: runs/detect/train12/train_batch0.jpg
Image Name: val_batch0_pred.jpg, Full Path: runs/detect/train12/val_batch0_pred.jpg
Image Name: PR_curve.png, Full Path: runs/detect/train12/PR_curve.png
Image Name: confusion_matrix_normalized.png, Full Path: runs/detect/train12/confusion_matrix_normalized.png
Image Name: P_curve.png, Full Path: runs/detect/train12/P_curve.png
Image Name: R_curve.png, Full Path: runs/detect/train12/R_curve.png
Image Name: val_batch0_labels.jpg, Full Path: runs/detect/train12/val_batch0_labels.jpg
Image Name: train_batch2.jpg, Full Path: runs/detect/train12/train_batch2.jpg
Image Name: confusion_matrix.png, Full Path: runs/detect/train12/confusion_matrix.png
Image Name: labels_correlogram.jpg, Full Path: runs/detect/train12/labels_correlogram.jpg
Image Name: train_batch1.jpg, Full Path: runs/detect/train12/train_batch1.jpg
Image Name: labels.jpg, Full Path:

In [4]:
#gets the path
path = x.save_dir
from pathlib import Path

pp = str(path)
print(type(pp))

<class 'str'>


In [69]:
model.ckpt['train_args']['data']

model.ckpt_path

model.ckpt['train_args']['name']

'train5'

In [16]:
import os
def find_image_file(folder, filename):
    extension = ['jpg','jpeg','png']
    for ext in extension:
        image_path = os.path.join(folder,f"{filename}.{ext}")
        if os.path.exists(image_path):
            return image_path
        
folder = "/home/arvind/Python/kubeflow/images"
filename = "frame_3"
x= find_image_file(folder, filename)

In [17]:
x

'/home/arvind/Python/kubeflow/images/frame_3.jpg'

In [22]:
os.path.basename(x).split('.')[1]

'jpg'

In [24]:
import shutil
shutil.copy(x,os.path.join("/home/arvind/Python","kubeflow"))

'/home/arvind/Python/kubeflow/frame_3.jpg'

In [37]:
from ultralytics import YOLO
from torch.utils.tensorboard import SummaryWriter
import os

project = "/home/arvind/Python/DL/kubeops"
name = "yolo"
epochs = 3

writer = SummaryWriter(log_dir=f"{project}/{name}")

def log_to_tensorboard(metrics, epoch):
    # Log any relevant metrics from YOLOv8 to TensorBoard
    writer.add_scalar('Loss/train', metrics['metrics_recall(B)'], epoch)
    writer.add_scalar('mAP/train', metrics['metrics_precision(B)'], epoch)
    # Add more metrics as needed, based on what YOLOv8 outputs
    writer.flush()

# Train the model and log metrics manually
results = model.train(data = "/home/arvind/Python/DL/datasets/datav1/v2/data.yaml", project = project, name = name, epochs = epochs, workers = 2, exist_ok = True)

# After each epoch, log to TensorBoard
for epoch, metrics in enumerate(results.results_dict):
    log_to_tensorboard(metrics, epoch)

# Close the TensorBoard writer after training
writer.close()




New https://pypi.org/project/ultralytics/8.3.4 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.85 🚀 Python-3.9.18 torch-2.4.0+cu121 CPU (AMD Ryzen 7 5825U with Radeon Graphics)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/home/arvind/Python/DL/datasets/datav1/v2/data.yaml, epochs=3, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=2, project=/home/arvind/Python/DL/kubeops, name=yolo, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, cl

train: Scanning /home/arvind/Python/DL/datasets/datav1/v2/train/labels.cache... 69 images, 0 backgrounds, 0 corrupt: 100%|██████████| 69/69 [00:00<?, ?it/s]
val: Scanning /home/arvind/Python/DL/datasets/datav1/v2/valid/labels.cache... 20 images, 0 backgrounds, 0 corrupt: 100%|██████████| 20/20 [00:00<?, ?it/s]

Plotting labels to /home/arvind/Python/DL/kubeops/yolo/labels.jpg... 


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
MLflow: logging run_id(aaf24128f38a44d785b1aead269f6864) to runs/mlflow
MLflow: view at http://127.0.0.1:5000 with 'mlflow server --backend-store-uri runs/mlflow'
MLflow: disable with 'yolo settings mlflow=False'
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to /home/arvind/Python/DL/kubeops/yolo
Starting training for 3 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/3         0G     0.6056      1.719     0.9284         24        640: 100%|██████████| 5/5 [00:24<00:00,  4.82s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.66s/it]

...

                   all         20         57     0.0872      0.806      0.617      0.532



3 epochs completed in 0.021 hours.
Optimizer stripped from /home/arvind/Python/DL/kubeops/yolo/weights/last.pt, 6.2MB
Optimizer stripped from /home/arvind/Python/DL/kubeops/yolo/weights/best.pt, 6.2MB

Validating /home/arvind/Python/DL/kubeops/yolo/weights/best.pt...
Ultralytics YOLOv8.2.85 🚀 Python-3.9.18 torch-2.4.0+cu121 CPU (AMD Ryzen 7 5825U with Radeon Graphics)
Model summary (fused): 168 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.41s/it]


                   all         20         57     0.0873      0.806      0.617      0.532
              employee          7          8    0.00471        0.5     0.0642      0.045
              security         19         27      0.139      0.963      0.897      0.819
        cleaning staff         18         22      0.118      0.955      0.891      0.732
Speed: 1.4ms preprocess, 51.6ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to /home/arvind/Python/DL/kubeops/yolo
MLflow: results logged to runs/mlflow
MLflow: disable with 'yolo settings mlflow=False'


AttributeError: 'DetMetrics' object has no attribute 'metrics'. See valid attributes below.

    Utility class for computing detection metrics such as precision, recall, and mean average precision (mAP) of an
    object detection model.

    Args:
        save_dir (Path): A path to the directory where the output plots will be saved. Defaults to current directory.
        plot (bool): A flag that indicates whether to plot precision-recall curves for each class. Defaults to False.
        on_plot (func): An optional callback to pass plots path and data when they are rendered. Defaults to None.
        names (dict of str): A dict of strings that represents the names of the classes. Defaults to an empty tuple.

    Attributes:
        save_dir (Path): A path to the directory where the output plots will be saved.
        plot (bool): A flag that indicates whether to plot the precision-recall curves for each class.
        on_plot (func): An optional callback to pass plots path and data when

In [38]:

results = model.train(data = "/home/arvind/Python/DL/datasets/datav1/v2/data.yaml", project = project, name = name, epochs = epochs, workers = 2, exist_ok = True)

New https://pypi.org/project/ultralytics/8.3.4 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.85 🚀 Python-3.9.18 torch-2.4.0+cu121 CPU (AMD Ryzen 7 5825U with Radeon Graphics)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/home/arvind/Python/DL/datasets/datav1/v2/data.yaml, epochs=3, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=2, project=/home/arvind/Python/DL/kubeops, name=yolo, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, cl

train: Scanning /home/arvind/Python/DL/datasets/datav1/v2/train/labels.cache... 69 images, 0 backgrounds, 0 corrupt: 100%|██████████| 69/69 [00:00<?, ?it/s]
val: Scanning /home/arvind/Python/DL/datasets/datav1/v2/valid/labels.cache... 20 images, 0 backgrounds, 0 corrupt: 100%|██████████| 20/20 [00:00<?, ?it/s]

Plotting labels to /home/arvind/Python/DL/kubeops/yolo/labels.jpg... 


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
MLflow: logging run_id(69fccaadc944426a9c20944acd1f2380) to runs/mlflow
MLflow: view at http://127.0.0.1:5000 with 'mlflow server --backend-store-uri runs/mlflow'
MLflow: disable with 'yolo settings mlflow=False'
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to /home/arvind/Python/DL/kubeops/yolo
Starting training for 3 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/3         0G     0.4957      1.159     0.8863         24        640: 100%|██████████| 5/5 [00:25<00:00,  5.02s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.78s/it]

...

        3/3         0G     0.5591      1.077     0.8979         31        640: 100%|██████████| 5/5 [00:22<00:00,  4.48s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.66s/it]

                   all         20         57      0.923       0.31      0.616      0.524



3 epochs completed in 0.022 hours.
Optimizer stripped from /home/arvind/Python/DL/kubeops/yolo/weights/last.pt, 6.2MB
Optimizer stripped from /home/arvind/Python/DL/kubeops/yolo/weights/best.pt, 6.2MB

Validating /home/arvind/Python/DL/kubeops/yolo/weights/best.pt...
Ultralytics YOLOv8.2.85 🚀 Python-3.9.18 torch-2.4.0+cu121 CPU (AMD Ryzen 7 5825U with Radeon Graphics)
Model summary (fused): 168 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.47s/it]


                   all         20         57      0.947      0.457      0.621      0.528
              employee          7          8          1          0     0.0723     0.0527
              security         19         27      0.913      0.773      0.895      0.809
        cleaning staff         18         22      0.929      0.598      0.897      0.723
Speed: 1.4ms preprocess, 53.1ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to /home/arvind/Python/DL/kubeops/yolo
MLflow: results logged to runs/mlflow
MLflow: disable with 'yolo settings mlflow=False'


In [40]:
results.results_dict

{'metrics/precision(B)': 0.9472937087093557,
 'metrics/recall(B)': 0.4571066854865929,
 'metrics/mAP50(B)': 0.6213614119959807,
 'metrics/mAP50-95(B)': 0.5282182468400969,
 'fitness': 0.5375325633556852}